In [22]:
# import os
import gc
import timm
import numpy as np
import pandas as pd
import sklearn.metrics
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts #, ReduceLROnPlateau, OneCycleLR, CosineAnnealingLR

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint #, EarlyStopping,BackboneFinetuning

import wandb
import librosa
import albumentations as A
from torchtoolbox.tools import mixup_data, mixup_criterion
import soundfile as sf
import pickle

import warnings
warnings.filterwarnings('ignore')


In [23]:
class Config:
    use_aug = False
    num_classes = 264
    batch_size = 64
    epochs = 2 # 12, 50
    PRECISION = 32    
    PATIENCE = 8    
    seed = 2023
    model = "tf_efficientnet_b0_ns"
    pretrained = True            
    weight_decay = 1e-3
    use_mixup = True
    mixup_alpha = 0.2   
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    

    
    SR = 32000
    DURATION = 5
    MAX_READ_SAMPLES = 5
    LR = 5e-4
    save_path =  "./exp1/"
    pickle_file_path = 'train_mel(dB,sr=32k,bin=128).pickle3'

In [24]:
pl.seed_everything(Config.seed, workers=True)

Global seed set to 2023


2023

In [25]:

def load_pickle(fname):
    f = open(fname, 'rb')
    out = pickle.load(f)
    f.close()
    return out


In [26]:
try:
    data = load_pickle(Config.pickle_file_path)  # see ogg2mel for how to convert ogg to mel_dict.pickle3
    bird_names, _ = np.unique(data['primary_label'], return_counts=True)
except FileNotFoundError:
    print("dataset not found! you can generate one by using ogg2mel.py")
    data=None
    exit(-1)


In [32]:
class BirdDataset(Dataset):
    def __init__(self, data, mode):
        assert mode in ['train', 'test', 'val'], f'invalid mode {mode}!, mode must be [train | val | test]'
        self.mode = mode

        self.sr = 32000
        self.duration = 5
        self.audio_length = self.duration * self.sr

        self.name_label_2_int_label = load_pickle("name_label_2_int_label.pickle3")  # a dict which saves mapping from


        if mode == 'train':
            total_data_length = len(data['primary_label'])
            self.data = {k: v[:int(0.8*total_data_length)] for k, v in data.items()}  # 0~80% as train set
        elif mode == 'val':
            total_data_length = len(data['primary_label'])
            self.data = {k: v[int(0.8 * total_data_length):] for k, v in data.items()}  # 80%~100% as val set
        elif mode == 'test':  # in test mode, data is in DataFrame form
            self.data_test_df = data
        else:
            raise ValueError(f'no such mode {mode}')

    @staticmethod
    def normalize(image):
        image = image / 255.0
        #image = torch.stack([image, image, image])
        return image

    def __getitem__(self, index):
        if self.mode == 'train' or  self.mode == 'val':
            mel = self.data['mel'][index]
            
            mel = self.crop_or_pad(mel)
            mel = self.mono_to_color(mel)
            mel = self.normalize(mel)
            mel = torch.from_numpy(mel).unsqueeze(0).float()
            
            label = self.name_label_2_int_label[self.data['primary_label'][index]]  # 'bird name' -> idx
            label = torch.nn.functional.one_hot(label, num_classes=264).float()
            # print(label)
            return mel, label

        elif self.mode == 'test':
            return self.read_file(self.data_test_df.loc[index, "path"])

    def __len__(self):
        if self.mode == 'train' or  self.mode == 'val':
            return len(self.data['primary_label'])
        elif self.mode == 'test':
            return len(self.data_test_df)

    def crop_or_pad(self, m, th=313):  # 313=5s*32000Hz/512
        length = m.shape[1]
        if length <= th: # pad short
            while m.shape[1] < th:  # repeat padding until th
                m = np.concatenate([m, m],axis=1)
            m = m[:,0:th]
        else:  # crop longer audio
            start = np.random.randint(length - th)
            m = m[:,start:start+th]
        return m #torch.from_numpy(m).unsqueeze(0)

    ########## following methods are for test use only ##################

    def audio_to_image(self, audio, ):
        melspec = librosa.feature.melspectrogram(y=audio, sr=self.sr, n_mels=128, fmin=0, fmax=self.sr//2)
        melspec = librosa.power_to_db(melspec).astype(np.float32)
        image = mono_to_color(melspec)
        # image = self.normalize(image)
        return melspec

    def read_file(self, filepath):
        audio, orig_sr = sf.read(filepath, dtype="float32")

        # if self.resample and orig_sr != self.sr:
        #     audio = librosa.resample(audio, orig_sr, self.sr, res_type=self.res_type)

        audios = []
        for i in range(self.audio_length, len(audio) + self.audio_length, self.audio_length):
            start = max(0, i - self.audio_length)
            end = start + self.audio_length
            audios.append(audio[start:end])

        if len(audios[-1]) < self.audio_length:
            audios = audios[:-1]

        images = [self.audio_to_image(audio) for audio in audios]
        images = np.stack(images)
        return images
    
    def mono_to_color(self, X, eps=1e-6, mean=None, std=None):
        mean = mean or X.mean()
        std = std or X.std()
        X = (X - mean) / (std + eps)
        
        _min, _max = X.min(), X.max()

        if (_max - _min) > eps:
            V = np.clip(X, _min, _max)
            V = 255 * (V - _min) / (_max - _min)
            V = V.astype(np.uint8)
        else:
            V = np.zeros_like(X, dtype=np.uint8)

        return V


In [33]:
train_dataset = BirdDataset(data=data, mode='train')
val_dataset = BirdDataset(data=data, mode='val')

train_dataloader = DataLoader(dataset=train_dataset, batch_size=Config.batch_size, shuffle=True, pin_memory=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=Config.batch_size, shuffle=False, pin_memory=True, drop_last=False)

In [34]:

def get_optimizer(lr, params):
    model_optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, params), 
            lr=lr,
            weight_decay=Config.weight_decay
        )
    interval = "epoch"
    
    lr_scheduler = CosineAnnealingWarmRestarts(
                            model_optimizer, 
                            T_0=Config.epochs, 
                            T_mult=1, 
                            eta_min=1e-6, 
                            last_epoch=-1
                        )

    return {
        "optimizer": model_optimizer, 
        "lr_scheduler": {
            "scheduler": lr_scheduler,
            "interval": interval,
            "monitor": "val_loss",
            "frequency": 1
        }
    }

In [37]:
class BirdClefModel(pl.LightningModule):
    def __init__(self, model_name=Config.model, num_classes = Config.num_classes, pretrained = Config.pretrained):
        super().__init__()
        self.num_classes = num_classes

        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if 'res' in model_name:
            self.in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Linear(self.in_features, num_classes)
        elif 'dense' in model_name:
            self.in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Linear(self.in_features, num_classes)
        elif 'efficientnet' in model_name:
            self.in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Sequential(
                nn.Linear(self.in_features, num_classes)
            )
        
        self.loss_function = nn.BCEWithLogitsLoss() 

    def forward(self,images):
        b,c,h,w = images.size()
        if c==1:
            images = images.repeat(1,3,1,1)
        
        logits = self.backbone(images)
        return logits
        
    def configure_optimizers(self):
        return get_optimizer(lr=Config.LR, params=self.parameters())

    def train_with_mixup(self, X, y):
        X, y_a, y_b, lam = mixup_data(X, y, alpha=Config.mixup_alpha)
        y_pred = self(X)
        loss_mixup = mixup_criterion(F.cross_entropy, y_pred, y_a, y_b, lam)
        return loss_mixup

    def training_step(self, batch, batch_idx):
        
        image, target = batch        
        print(torch.max(image), torch.min(image), '--=-=-=-=-=========-=-')
        if Config.use_mixup:
            loss = self.train_with_mixup(image, target)
        else:
            y_pred = self(image)
            loss = self.loss_function(y_pred,target)

        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss        

    def validation_step(self, batch, batch_idx):
        image, target = batch     
        y_pred = self(image)
        val_loss = self.loss_function(y_pred, target)
        
        self.validation_step_outputs = {"val_loss": val_loss, "logits": y_pred, "targets": target}
        self.log("val_loss", val_loss)

        return self.validation_step_outputs
    
    def train_dataloader(self):
        return self._train_dataloader 
    
    def validation_dataloader(self):
        return self._validation_dataloader
    
    def on_validation_epoch_end(self):

        avg_loss = self.validation_step_outputs['val_loss'].mean()
        output_val = self.validation_step_outputs['logits'].sigmoid().cpu().detach().numpy()
        target_val = self.validation_step_outputs['targets'].cpu().detach().numpy()

        avg_score = sklearn.metrics.label_ranking_average_precision_score(target_val,output_val)

        self.log('val_accuracy', avg_score)
        self.validation_step_outputs.clear()  # free memory
        
        return {'val_loss': avg_loss,'val_cmap':avg_score}    
    
    
    

In [38]:
# define the logger
wandb_logger = WandbLogger(project='Bird2023', log_model="all", name='efficientnet_b0_epoch_50')
logger = wandb_logger



# define the model
audio_model = BirdClefModel()


# early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.00, patience=Config.PATIENCE, verbose= True, mode="min")
checkpoint_callback = ModelCheckpoint(dirpath=Config.save_path,
                                        save_top_k=1,
                                        save_last= True,
                                        save_weights_only=False,
                                        filename= f'./{Config.model}_loss',
                                        verbose= True,
                                        monitor='val_accuracy',
                                        mode='max',
                                        auto_insert_metric_name = True)

callbacks_to_use = [checkpoint_callback]#,early_stop_callback]

# define the trainer
trainer = Trainer(
    val_check_interval=0.5,
    deterministic=True,
    max_epochs=Config.epochs,
    logger=logger,   
    callbacks=callbacks_to_use,
    precision=Config.PRECISION, accelerator="gpu",
    devices=[7],auto_select_gpus=True,
    num_sanity_val_steps=0 
)

# train the model
trainer.fit(audio_model, train_dataloaders = train_dataloader, val_dataloaders = val_dataloader)                


# close the wandb run and free memory
wandb.finish()
gc.collect()
torch.cuda.empty_cache()


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]



  | Name          | Type              | Params
----------------------------------------------------
0 | backbone      | EfficientNet      | 4.3 M 
1 | loss_function | BCEWithLogitsLoss | 0     
----------------------------------------------------
4.3 M     Trainable params
0         Non-trainable params
4.3 M     Total params
17.383    Total estimated model params size (MB)


Training: -1it [00:00, ?it/s]

tensor(1., device='cuda:7') tensor(0., device='cuda:7') --=-=-=-=-=========-=-
tensor(1., device='cuda:7') tensor(0., device='cuda:7') --=-=-=-=-=========-=-
tensor(1., device='cuda:7') tensor(0., device='cuda:7') --=-=-=-=-=========-=-
tensor(1., device='cuda:7') tensor(0., device='cuda:7') --=-=-=-=-=========-=-
tensor(1., device='cuda:7') tensor(0., device='cuda:7') --=-=-=-=-=========-=-
tensor(1., device='cuda:7') tensor(0., device='cuda:7') --=-=-=-=-=========-=-
tensor(1., device='cuda:7') tensor(0., device='cuda:7') --=-=-=-=-=========-=-
tensor(1., device='cuda:7') tensor(0., device='cuda:7') --=-=-=-=-=========-=-
tensor(1., device='cuda:7') tensor(0., device='cuda:7') --=-=-=-=-=========-=-
tensor(1., device='cuda:7') tensor(0., device='cuda:7') --=-=-=-=-=========-=-
tensor(1., device='cuda:7') tensor(0., device='cuda:7') --=-=-=-=-=========-=-
tensor(1., device='cuda:7') tensor(0., device='cuda:7') --=-=-=-=-=========-=-
tensor(1., device='cuda:7') tensor(0., device='cuda: